In [9]:
!nvidia-smi

Thu Jan 18 00:31:45 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   30C    P0    38W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES']

'2,3'

In [10]:



!cd ../../FastChat/fastchat/llm_judge && CUDA_VISIBLE_DEVICES=2 python gen_model_answer.py \
    --model-path /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/results/baselines/HuggingFaceH4/zephyr-7b-beta \
    --model-id zephyr-7b \
    --question-begin 0 \
    --question-end 1 \
    --max-new-token 256 \




[2024-01-18 00:31:52,025] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Output to data/mt_bench/model_answer/zephyr-7b.jsonl
  0%|                                                     | 0/1 [00:00<?, ?it/s]/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/transformers/src/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
100%|█████████████████████████████████████████████| 1/1 [00:21<00:00, 21.96s/it]


In [14]:
!cd ../../FastChat/fastchat/llm_judge && \
    python gen_judgment.py --model-list zephyr-7b --parallel 1 --first-n 1

^C
Traceback (most recent call last):
  File "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/FastChat/fastchat/llm_judge/gen_judgment.py", line 12, in <module>
    from fastchat.llm_judge.common import (
  File "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/FastChat/fastchat/llm_judge/common.py", line 15, in <module>
    import anthropic
  File "/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/anthropic/__init__.py", line 3, in <module>
    from . import types
  File "/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/anthropic/types/__init__.py", line 5, in <module>
    from .completion import Completion as Completion
  File "/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/anthropic/types/completion.py", line 5, in <module>
    from .._models import BaseModel
  File "/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-p

In [1]:
from rosemary import jpt_setup; jpt_setup()

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [5]:
import os
from note_pruning_analysis import open_instruct_dir


question_file = os.path.join(open_instruct_dir, 'external', 'FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl')
question_file


'/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl'

In [18]:
import torch

# model_id = 'llama-7b'
# model_path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/huggyllama/llama-7b'
model_id = 'tulu'
model_id = 'zephyr-7b'
model_path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/results/baselines/HuggingFaceH4/zephyr-7b-beta'

torch_dtype = 'float16'
question_file = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl'
answer_file = f'{model_id}.jsonl'


kwargs = {
    'model_path': model_path,
    'model_id': model_id,
    'question_file': question_file,
    'question_begin': 0,
    'question_end': 1,
    'answer_file': answer_file,
    'max_new_token': 256,
    'num_choices': 1,
    'num_gpus_per_model': 1,
    'num_gpus_total': 1,
    'max_gpu_memory': None,
    'dtype': getattr(torch, torch_dtype),
    'revision': 'main',
}


run_eval(**kwargs)

for k, v in kwargs.items():
    globals()[k] = v


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB (GPU 0; 31.75 GiB total capacity; 1002.02 MiB already allocated; 441.88 MiB free; 1020.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
from fastchat.llm_judge.common import load_questions, temperature_config
from fastchat.model import load_model, get_conversation_template
from fastchat.llm_judge.gen_model_answer import get_model_answers, run_eval



# questions = load_questions(question_file, question_begin, question_end)
questions = load_questions(question_file, None, None)
# # random shuffle the questions to balance the loading
# random.shuffle(questions)
questions


[{'question_id': 81,
  'category': 'writing',
  'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
   'Rewrite your previous response. Start every sentence with the letter A.']},
 {'question_id': 82,
  'category': 'writing',
  'turns': ["Draft a professional email seeking your supervisor's feedback on the 'Quarterly Financial Report' you prepared. Ask specifically about the data analysis, presentation style, and the clarity of conclusions drawn. Keep the email short and to the point.",
   'Take a moment to evaluate and critique your own response.']},
 {'question_id': 83,
  'category': 'writing',
  'turns': ['Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words.',

In [15]:
from collections import Counter

Counter([x['category'] for x in questions])

Counter({'writing': 10,
         'roleplay': 10,
         'reasoning': 10,
         'math': 10,
         'coding': 10,
         'extraction': 10,
         'stem': 10,
         'humanities': 10})

In [11]:

# Split the question file into `num_gpus` files
assert num_gpus_total % num_gpus_per_model == 0
use_ray = num_gpus_total // num_gpus_per_model > 1

if use_ray:
    get_answers_func = ray.remote(num_gpus=num_gpus_per_model)(
        get_model_answers
    ).remote
else:
    get_answers_func = get_model_answers

In [12]:
chunk_size = len(questions) // (num_gpus_total // num_gpus_per_model)
chunk_size

80

In [13]:
model, tokenizer = load_model(
    model_path,
    revision=revision,
    device="cuda",
    num_gpus=num_gpus_per_model,
    max_gpu_memory=max_gpu_memory,
    dtype=dtype,
    load_8bit=False,
    cpu_offloading=False,
    debug=False,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB (GPU 0; 31.75 GiB total capacity; 1002.02 MiB already allocated; 441.88 MiB free; 1020.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

from tqdm import tqdm

for question in tqdm(questions):
    if question["category"] in temperature_config:
        temperature = temperature_config[question["category"]]
    else:
        temperature = 0.7

    choices = []
    for i in range(num_choices):
        torch.manual_seed(i)
        conv = get_conversation_template(model_id)
        turns = []
        for j in range(len(question["turns"])):
            qs = question["turns"][j]
            conv.append_message(conv.roles[0], qs)
            conv.append_message(conv.roles[1], None)
            prompt = conv.get_prompt()
            input_ids = tokenizer([prompt]).input_ids

            if temperature < 1e-4:
                do_sample = False
            else:
                do_sample = True

            # some models may error out when generating long outputs
            try:
                output_ids = model.generate(
                    torch.as_tensor(input_ids).cuda(),
                    do_sample=do_sample,
                    temperature=temperature,
                    max_new_tokens=max_new_token,
                )
                if model.config.is_encoder_decoder:
                    output_ids = output_ids[0]
                else:
                    output_ids = output_ids[0][len(input_ids[0]) :]

                # be consistent with the template's stop_token_ids
                if conv.stop_token_ids:
                    stop_token_ids_index = [
                        i
                        for i, id in enumerate(output_ids)
                        if id in conv.stop_token_ids
                    ]
                    if len(stop_token_ids_index) > 0:
                        output_ids = output_ids[: stop_token_ids_index[0]]

                output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
                if conv.stop_str and isinstance(conv.stop_str, list):
                    stop_str_indices = sorted(
                        [
                            output.find(stop_str)
                            for stop_str in conv.stop_str
                            if output.find(stop_str) > 0
                        ]
                    )
                    if len(stop_str_indices) > 0:
                        output = output[: stop_str_indices[0]]
                elif conv.stop_str and output.find(conv.stop_str) > 0:
                    output = output[: output.find(conv.stop_str)]

                for special_token in tokenizer.special_tokens_map.values():
                    if isinstance(special_token, list):
                        for special_tok in special_token:
                            output = output.replace(special_tok, "")
                    else:
                        output = output.replace(special_token, "")
                output = output.strip()

                if conv.name == "xgen" and output.startswith("Assistant:"):
                    output = output.replace("Assistant:", "", 1).strip()
            except RuntimeError as e:
                print("ERROR question ID: ", question["question_id"])
                output = "ERROR"

            conv.update_last_message(output)
            turns.append(output)

        choices.append({"index": i, "turns": turns})

#     # Dump answers
#     os.makedirs(os.path.dirname(answer_file), exist_ok=True)
#     with open(os.path.expanduser(answer_file), "a") as fout:
#         ans_json = {
#             "question_id": question["question_id"],
#             "answer_id": shortuuid.uuid(),
#             "model_id": model_id,
#             "choices": choices,
#             "tstamp": time.time(),
#         }
#         fout.write(json.dumps(ans_json) + "\n")
